In [ ]:
import pandas as pd
import numpy as np
import os

### Loading and Pivoting

In [ ]:
base_path = os.path.dirname(__file__) + '//Data//'

# Loading data
domestic_consumption = pd.read_csv(base_path + 'Coffee_domestic_consumption.csv')
coffee_production = pd.read_csv(base_path + 'Coffee_production.csv')
coffee_inventory = pd.read_csv(base_path + 'Coffee_green_coffee_inventorie.csv')

# Pivoting
domestic_consumption = pd.wide_to_long(domestic_consumption, ["Consumption"], i="country", j="year")